In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from numpy.random import seed
from numpy.random import randn
from numpy import percentile

In [ ]:
#implementation 
def compute_mse_loss(y, tx, w):
    """ return mse loss """
    e = y - np.dot(tx,w)
    return 1/2 * np.mean(e**2)


def compute_gradient(y, tx, w):
    """Compute the gradient."""
    N = len(y)
    e = y - np.dot(tx,w)
    grad = -(np.dot(tx.T,e))/N
    return grad


def sigmoid(x):
    """sigmoid function """
    return 1.0 / (1 + np.exp(-x))


def compute_lg_loss(y, x, w):
    e = 1e-11
    p = sigmoid(np.dot(x, w))
    loss = -np.dot(y, np.log(p + e)) - np.dot((1 - y).T, np.log(1 - p + e))
    return loss


def compute_lg_grad(y, x, w):
    p = sigmoid(np.dot(x, w))
    grad = np.dot(x.T, (p - y))
    return grad


def least_squares_GD(y, tx, initial_w, max_iters, gamma):

    losses = []
    w = initial_w
    threshold = 1e-8
    for n_iter in range(max_iters):

        grad = compute_gradient(y, tx, w)
        grad = grad/np.linalg.norm(grad)

        loss = compute_mse_loss(y, tx, w)

        w = w - gamma*grad
        # store loss in order to check the convergence
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1]-losses[-2]) < threshold:
            break

    return w,losses



def least_squares_SGD(y, tx, initial_w, max_iters, gamma):

    losses = []
    w = initial_w
    threshold = 1e-8
    batch_size = 1

    for n_iter in range(max_iters):

        random_num = np.random.randint(0, len(y), size=batch_size)
        r_y = y[random_num]
        r_tx = tx[random_num]

        grad = compute_gradient(r_y, r_tx, w)
        grad = grad/np.linalg.norm(grad)

        w = w - gamma*grad

        # store loss in order to check the convergence
        loss = compute_mse_loss(y, tx, w)
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1]-losses[-2]) < threshold:
            break

    return w,losses




def least_squares(y, tx):
    """calculate the least squares solution."""
    XTX = np.dot(tx.T,tx)
    XTY = np.dot(tx.T,y)
    w = np.linalg.solve(XTX, XTY)
    loss = compute_mse_loss(y, tx, w)

    return w,loss


def ridge_regression(y, tx, lambda_):

    reg = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    XTX = np.dot(tx.T,tx) + reg
    XTY = np.dot(tx.T,y)
    w = np.linalg.solve(XTX, XTY)
    loss = compute_mse_loss(y, tx, w)

    return w,loss

# Logistic regression
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    w = initial_w
    losses = []
    threshold = 1e-8

    for i in range(max_iters):
        grad = compute_lg_grad(y, tx, w)
        grad = grad / np.linalg.norm(grad)

        w = w - gamma * grad
        # store loss in order to check the convergence
        loss = compute_lg_loss(y, tx, w)
        # print(loss)
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1]-losses[-2]) < threshold:
            break

    return w, losses

def reg_logistic_regression(y, tx, lambda_ , initial_w, max_iters, gamma):


    w = initial_w
    losses = []
    threshold = 1e-8

    for i in range(max_iters):

        grad = compute_lg_grad(y, tx, w)
        grad =  grad + 2*lambda_*w

        grad = grad / np.linalg.norm(grad)

        w = w - gamma * grad

        # store loss in order to check the convergence
        loss = compute_lg_loss(y, tx, w)
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break

    return w, losses

In [ ]:


# divide data into three groups
def divide_data(tx,y):

    grp_1_tx = tx[tx[:, 22] >= 2]
    def_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29]
    grp_1_tx = grp_1_tx[:, def_1]
    
    grp_2_tx = tx[tx[:, 22] == 1]
    def_2 = [0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 29]
    grp_2_tx = grp_2_tx[:, def_2]

    grp_3_tx = tx[tx[:, 22] == 0]
    def_3 = [0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21]
    grp_3_tx = grp_3_tx[:, def_3]

    grp_1_y = y[tx[:, 22] >= 2]
    grp_2_y = y[tx[:, 22] == 1]
    grp_3_y = y[tx[:, 22] == 0]

    return grp_1_tx, grp_2_tx, grp_3_tx, grp_1_y, grp_2_y, grp_3_y, def_1, def_2, def_3

# normalize data
def normalize(tX):
    for i in range(tX.shape[1]):
        if (tX[:,i] == 1).all(): #it's not necessary to normalize the bais term
            continue
        tX[:, i] = (tX[:, i] - min(tX[:, i])) / (max(tX[:, i] - min(tX[:, i])))

    return tX

def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

def build_tx_poly(grp_tX, degree1):

    if grp_tX.shape[1] == 30:
        grp_tX_poly_der = build_poly(grp_tX[:,:13], degree1)
        grp_tX_poly_pri = grp_tX[:,13:]
        grp_tX_poly = np.c_[grp_tX_poly_der, grp_tX_poly_pri]
    else:
        grp_tX_poly_der = build_poly(grp_tX[:,:10], degree1)
        grp_tX_poly_pri = grp_tX[:,10:]
        grp_tX_poly = np.c_[grp_tX_poly_der, grp_tX_poly_pri]
    return grp_tX_poly

In [ ]:
def split_data(x, y, k):
    """split the dataset based on the split ratio."""
    # generate random indices

    data_size = len(y)
    indices = np.random.permutation(data_size)
    cros_val_set = []
    label_set = []

    for i in range(k):
        # ratio = i/k
        fold = x[indices[int(i / k * data_size):int((i + 1) / k * data_size)]]
        label = y[indices[int(i / k * data_size):int((i + 1) / k * data_size)]]
        cros_val_set.append(fold)
        label_set.append(label)

    return cros_val_set, label_set

def build_combinations(k):

    folds_id = set()
    leave_one_out = set()
    combinations = []
    for i in range(k):
        folds_id.add(i)
    for i in range(k):
        leave_one_out.add(i)
        combinations.append(folds_id.difference(leave_one_out))
        leave_one_out = set()

    return combinations,folds_id

In [ ]:
def clean_outlier(tX):
    for i in range(tX.shape[1]):
        # calculate summary statistics
        data = tX[:,i]
        data_mean, data_std = np.mean(data), np.std(data)
        # identify outliers
        cut_off = data_std * 3
        lower, upper = data_mean - cut_off, data_mean + cut_off
        outliers_removed = [x for x in data if x >= lower and x <= upper]
        value_rep_lo = np.min(outliers_removed)
        value_rep_up = np.max(outliers_removed)
        idx_outlier_lo = np.where((data < lower))
        idx_outlier_up = np.where((data > upper))
        data[idx_outlier_lo] = value_rep_lo
        data[idx_outlier_up] = value_rep_up
        tX[:,i] = data
    return tX

In [ ]:
#load data
from proj1_helpers import *
# TODO: download train data and supply path here 
DATA_TRAIN_PATH = '/Users/eddie/Desktop/Machine Learning CS-433/ML_course/projects/project1/dataset/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
tX[tX[:,0]==-999] = np.median(tX[tX[:,0]!=-999,0])

In [ ]:
grp_1_tx, grp_2_tx, grp_3_tx, grp_1_y, grp_2_y, grp_3_y, def_1, def_2, def_3 = divide_data(tX,y)

In [ ]:
# the training data has been devided into three groups 
# and before testing each group by algorithm defined above,
# we process the raw data by cleaning the outliers, 
# adding polynomial basis and normalization  

In [ ]:
grp_1_tx = clean_outlier(grp_1_tx)
grp_2_tx = clean_outlier(grp_2_tx)
grp_3_tx = clean_outlier(grp_3_tx)

In [ ]:

degree1 = 9
degree2 = 9
degree3 = 9

grp_1_tx_poly = build_tx_poly(grp_1_tx, degree1)
grp_2_tx_poly = build_tx_poly(grp_2_tx, degree2)
grp_3_tx_poly = build_tx_poly(grp_3_tx, degree3)

grp_1_tx_poly = normalize(grp_1_tx_poly)
grp_2_tx_poly = normalize(grp_2_tx_poly)
grp_3_tx_poly = normalize(grp_3_tx_poly)

In [ ]:
#only for logistic regression and regularized logistic regression
# grp_1_y[grp_1_y==-1] = 0
# grp_2_y[grp_2_y==-1] = 0
# grp_3_y[grp_3_y==-1] = 0

In [ ]:
#here we can decide which data group we would like to test based on cross validation

In [ ]:
# cros_val_set, label_set = split_data(grp_1_tx_poly, grp_1_y, 5)
# cros_val_set, label_set = split_data(grp_2_tx_poly, grp_2_y, 5)
# cros_val_set, label_set = split_data(grp_3_tx_poly, grp_3_y, 5)

combinations,folds_id = build_combinations(5)

In [ ]:
def validate(cros_val_set, label_set, combinations,folds_id):

    loss = 0;
    count = 0;
    num = 0;
    for combination in combinations:
        tr = []
        tr_l = []
        te_id = list(folds_id.difference(combination))[0]
        te = cros_val_set[te_id]
        te_l = label_set[te_id]
        for fold_id in combination:
            tr.append(cros_val_set[fold_id])
            tr_l.append(label_set[fold_id])
        tr = np.vstack(tr)
        tr_l = np.hstack(tr_l)
        
        #los is used to check the convergence 
        #w, los = least_squares_SGD(tr_l, tr, initial_w, 1800, 0.03)
        #w, los = least_squares_GD(tr_l, tr, initial_w, 500, 0.03)
        #w, los = least_squares(tr_l, tr)
        #w, los = ridge_regression(tr_l, tr, 0.00000000001)
        #w, los = logistic_regression(tr_l, tr, np.zeros(grp_2_tx_poly.shape[1]), 10000, 0.1)
        #w, los = reg_logistic_regression(tr_l, tr, 0.000000001 , np.zeros(grp_3_tx_poly.shape[1]), 10000, 0.02)
        
        pred_l = predict_labels(w, te)
        
        #only for logistic regression and regularized logistic regression
        #pred_l [pred_l==-1] =0
        
        loss += len(te_l) - sum(te_l == pred_l)
        count += sum(te_l == pred_l)
    
        num += len(te_l)
    loss = loss / len(folds_id) #this loss is the number of wrong predicitions
    acc = count / num
    return loss, acc, los

In [ ]:
#by selecting different methods and parameters in the validate function, 
#we can test their performance on different data group separately

In [ ]:
initial_w = np.zeros(grp_3_tx_poly.shape[1])

In [ ]:
loss,acc,los = validate(cros_val_set, label_set, combinations,folds_id)

In [ ]:
loss

In [ ]:
acc